In [1]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

sc = SparkContext()
sqlContext = SQLContext(sc)

21/09/23 10:51:18 WARN Utils: Your hostname, lukas-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
21/09/23 10:51:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/23 10:51:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Read data from ``data/hotels_bahamas_min.csv``

Read minimal data about hotels in the Bahamas from the CSV file an store them in temporary table ``hotels``.

In [2]:
df = sqlContext.read.format('csv').options(header='true', inferschema='true').load('/home/lukas/spark/data/hotels_bahamas_min.csv')
df.registerTempTable("hotels")
sqlContext.sql("SELECT * FROM hotels").show()
sqlContext.sql("SELECT * FROM hotels SKYLINE OF price MIN, distance MIN").show()

+---+-----+--------+
| id|price|distance|
+---+-----+--------+
|  1|  257|     600|
|  2|  323|     350|
|  3|  276|       0|
|  4|  234|     100|
|  5|  265|     250|
|  6|  257|     500|
+---+-----+--------+

+---+-----+--------+
| id|price|distance|
+---+-----+--------+
|  3|  276|       0|
|  4|  234|     100|
+---+-----+--------+



In [3]:
sqlContext.sql("SELECT * FROM hotels").explain("extended")
sqlContext.sql("SELECT * FROM hotels SKYLINE OF price MIN, distance MIN").explain()
sqlContext.sql("SELECT * FROM hotels SKYLINE OF price MIN, distance MIN").explain("extended")

== Parsed Logical Plan ==
'Project [*]
+- 'UnresolvedRelation [hotels], [], false

== Analyzed Logical Plan ==
id: int, price: int, distance: int
Project [id#16, price#17, distance#18]
+- SubqueryAlias hotels
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
FileScan csv [id#16,price#17,distance#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/lukas/spark/data/hotels_bahamas_min.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:int,price:int,distance:int>

== Physical Plan ==
BlockNestedLoopIncompleteSkyline SkylineIsNotDistinct, [price#17 MIN, distance#18 MIN]
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#84]
   +- BlockNestedLoopSkyline SkylineIsNotDistinct, [price#17 MIN, distance#18 MIN], ArrayBuffer(isnotnull(price#17), isnotnull(distance#18)), true
      +- Exchange hashpartitioning(isnotnull(price#17), isnotnull(distance#18)

In [4]:
sqlContext.sql("SELECT * FROM hotels").explain("extended")
sqlContext.sql("SELECT * FROM hotels COMPLETE SKYLINE OF price MIN, distance MIN").explain()
sqlContext.sql("SELECT * FROM hotels COMPLETE SKYLINE OF price MIN, distance MIN").explain("extended")

== Parsed Logical Plan ==
'Project [*]
+- 'UnresolvedRelation [hotels], [], false

== Analyzed Logical Plan ==
id: int, price: int, distance: int
Project [id#16, price#17, distance#18]
+- SubqueryAlias hotels
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
FileScan csv [id#16,price#17,distance#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/home/lukas/spark/data/hotels_bahamas_min.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:int,price:int,distance:int>

== Physical Plan ==
BlockNestedLoopIncompleteSkyline SkylineIsNotDistinct, [price#17 MIN, distance#18 MIN]
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#129]
   +- BlockNestedLoopSkyline SkylineIsNotDistinct, [price#17 MIN, distance#18 MIN], ArrayBuffer(isnotnull(price#17), isnotnull(distance#18)), true
      +- Exchange hashpartitioning(isnotnull(price#17), isnotnull(distance#18

In [5]:
df.filter("price > 200").skyline(["price", "distance"], minMaxDiff=["min", "min"]).show()
df.filter("price > 200").skyline(["price", "distance"], minMaxDiff=["min", "min"]).explain("extended")

+---+-----+--------+
| id|price|distance|
+---+-----+--------+
|  3|  276|       0|
|  4|  234|     100|
+---+-----+--------+

== Parsed Logical Plan ==
'SkylineOperator SkylineIsNotDistinct, SkylineUnspecifiedCompleteness, ['price MIN, 'distance MIN]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Analyzed Logical Plan ==
id: int, price: int, distance: int
SkylineOperator SkylineIsNotDistinct, SkylineUnspecifiedCompleteness, [price#17 MIN, distance#18 MIN]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
SkylineOperator SkylineIsNotDistinct, SkylineUnspecifiedCompleteness, [price#17 MIN, distance#18 MIN]
+- Filter (isnotnull(price#17) AND (price#17 > 200))
   +- Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
BlockNestedLoopIncompleteSkyline SkylineIsNotDistinct, [price#17 MIN, distance#18 MIN]
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#237]
   +- BlockNestedLoopSkyline Skyli

In [6]:
df.filter("price > 200").skylineDistinct(["price", "distance"], minMaxDiff=["min", "min"]).show()
df.filter("price > 200").skylineDistinct(["price", "distance"], minMaxDiff=["min", "min"]).explain("extended")

+---+-----+--------+
| id|price|distance|
+---+-----+--------+
|  3|  276|       0|
|  4|  234|     100|
+---+-----+--------+

== Parsed Logical Plan ==
'SkylineOperator SkylineIsDistinct, SkylineUnspecifiedCompleteness, ['price MIN, 'distance MIN]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Analyzed Logical Plan ==
id: int, price: int, distance: int
SkylineOperator SkylineIsDistinct, SkylineUnspecifiedCompleteness, [price#17 MIN, distance#18 MIN]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
SkylineOperator SkylineIsDistinct, SkylineUnspecifiedCompleteness, [price#17 MIN, distance#18 MIN]
+- Filter (isnotnull(price#17) AND (price#17 > 200))
   +- Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
BlockNestedLoopIncompleteSkyline SkylineIsDistinct, [price#17 MIN, distance#18 MIN]
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#328]
   +- BlockNestedLoopSkyline SkylineIsDistinct

In [7]:
df.filter("price > 200").skyline(df.price.smin(), df.distance.smax()).explain("extended")
df.filter("price > 200").skylineDistinct(df.price.smin(), df.distance.smax()).explain("extended")

== Parsed Logical Plan ==
SkylineOperator SkylineIsNotDistinct, SkylineUnspecifiedCompleteness, [price#17 MIN, distance#18 MAX]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Analyzed Logical Plan ==
id: int, price: int, distance: int
SkylineOperator SkylineIsNotDistinct, SkylineUnspecifiedCompleteness, [price#17 MIN, distance#18 MAX]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
SkylineOperator SkylineIsNotDistinct, SkylineUnspecifiedCompleteness, [price#17 MIN, distance#18 MAX]
+- Filter (isnotnull(price#17) AND (price#17 > 200))
   +- Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
BlockNestedLoopIncompleteSkyline SkylineIsNotDistinct, [price#17 MIN, distance#18 MAX]
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#366]
   +- BlockNestedLoopSkyline SkylineIsNotDistinct, [price#17 MIN, distance#18 MAX], ArrayBuffer(isnotnull(price#17), isnotnull(distance#18)), true
      +- Ex

In [8]:
df.filter("price > 200").skylineComplete(df.price.smin(), df.distance.smax()).explain("extended")
df.filter("price > 200").skylineDistinctComplete(df.price.smin(), df.distance.smax()).explain("extended")

== Parsed Logical Plan ==
SkylineOperator SkylineIsNotDistinct, SkylineIsComplete, [price#17 MIN, distance#18 MAX]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Analyzed Logical Plan ==
id: int, price: int, distance: int
SkylineOperator SkylineIsNotDistinct, SkylineIsComplete, [price#17 MIN, distance#18 MAX]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
SkylineOperator SkylineIsNotDistinct, SkylineIsComplete, [price#17 MIN, distance#18 MAX]
+- Filter (isnotnull(price#17) AND (price#17 > 200))
   +- Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
BlockNestedLoopSkyline SkylineIsNotDistinct, [price#17 MIN, distance#18 MAX], List(), false
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#438]
   +- BlockNestedLoopSkyline SkylineIsNotDistinct, [price#17 MIN, distance#18 MAX], false
      +- *(1) Filter (isnotnull(price#17) AND (price#17 > 200))
         +- FileScan csv [id#16,price#

In [9]:
df.filter("price > 200").skylineComplete(["price", "distance"], minMaxDiff=["min", "min"]).show()
df.filter("price > 200").skylineComplete(["price", "distance"], minMaxDiff=["min", "min"]).explain("extended")
df.filter("price > 200").skylineDistinctComplete(["price", "distance"], minMaxDiff=["min", "min"]).show()
df.filter("price > 200").skylineDistinctComplete(["price", "distance"], minMaxDiff=["min", "min"]).explain("extended")

+---+-----+--------+
| id|price|distance|
+---+-----+--------+
|  3|  276|       0|
|  4|  234|     100|
+---+-----+--------+

== Parsed Logical Plan ==
'SkylineOperator SkylineIsNotDistinct, SkylineIsComplete, ['price MIN, 'distance MIN]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Analyzed Logical Plan ==
id: int, price: int, distance: int
SkylineOperator SkylineIsNotDistinct, SkylineIsComplete, [price#17 MIN, distance#18 MIN]
+- Filter (price#17 > 200)
   +- Relation[id#16,price#17,distance#18] csv

== Optimized Logical Plan ==
SkylineOperator SkylineIsNotDistinct, SkylineIsComplete, [price#17 MIN, distance#18 MIN]
+- Filter (isnotnull(price#17) AND (price#17 > 200))
   +- Relation[id#16,price#17,distance#18] csv

== Physical Plan ==
BlockNestedLoopSkyline SkylineIsNotDistinct, [price#17 MIN, distance#18 MIN], List(), false
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#525]
   +- BlockNestedLoopSkyline SkylineIsNotDistinct, [price#17 MIN, di